# 02: Sensitivity Analysis with TSM

**Difficulty:** Intermediate | **Time:** ~15 minutes

## Overview

Learn how to perform parameter sensitivity analysis using Time-Separated Mechanics (TSM).

### What You'll Learn

- How TSM computes analytical sensitivities
- Visualizing parameter influence over time
- Identifying most influential parameters
- Comparing TSM with finite differences

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.config import CONFIG, get_theta_true
from src.tsm import compute_tsm_sensitivity
from src.solvers import solve_biofilm

np.random.seed(42)
output_dir = Path('figures')
output_dir.mkdir(exist_ok=True)

print("✓ Setup complete")

## 1. Compute TSM Sensitivities

TSM computes ∂y/∂θ analytically, avoiding expensive finite differences.

In [ ]:
theta_ref = get_theta_true()[0:5]  # M1 parameters

print("Computing TSM sensitivities...")
S = compute_tsm_sensitivity(theta_ref, CONFIG["M1"])

print(f"✓ Sensitivity matrix shape: {S.shape}")
print(f"  (n_outputs={S.shape[0]}, n_parameters={S.shape[1]})")

## 2. Visualize Sensitivities Over Time

In [ ]:
t = np.linspace(0, CONFIG["M1"]["maxtimestep"] * CONFIG["M1"]["dt"], S.shape[0])
param_names = ['a₁₁', 'a₁₂', 'a₂₂', 'b₁', 'b₂']

fig, axes = plt.subplots(5, 1, figsize=(10, 12))

for i, (ax, name) in enumerate(zip(axes, param_names)):
    ax.plot(t, S[:, i], linewidth=2)
    ax.set_ylabel(f'∂y/∂{name}', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.set_title(f'Sensitivity to {name}', fontsize=11)

axes[-1].set_xlabel('Time', fontsize=12)
plt.tight_layout()
plt.savefig(output_dir / '02_sensitivity_time.png', dpi=150)
plt.show()

print("✓ Sensitivity plot saved")

## 3. Parameter Importance Ranking

In [ ]:
# Compute total absolute sensitivity
total_sensitivity = np.sum(np.abs(S), axis=0)
importance_ranking = np.argsort(total_sensitivity)[::-1]

print("Parameter Importance Ranking:")
print("=" * 50)
for rank, idx in enumerate(importance_ranking, 1):
    print(f"{rank}. {param_names[idx]:<6} : {total_sensitivity[idx]:>10.4f}")
print("=" * 50)

# Bar plot
fig, ax = plt.subplots(figsize=(8, 5))
colors = plt.cm.viridis(np.linspace(0, 1, 5))
bars = ax.bar(param_names, total_sensitivity, color=colors, edgecolor='black')
ax.set_ylabel('Total Absolute Sensitivity', fontsize=12)
ax.set_title('Parameter Importance', fontsize=14)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(output_dir / '02_importance.png', dpi=150)
plt.show()

## 4. Finite Difference Comparison

Validate TSM against finite differences.

In [ ]:
print("Computing finite difference sensitivities (this is slower)...")

eps = 1e-5
config_short = CONFIG["M1"].copy()
config_short["maxtimestep"] = 10  # Use shorter simulation

# Baseline
y_base = solve_biofilm(theta_ref, config_short)

# Finite differences
S_fd = np.zeros((len(y_base), len(theta_ref)))
for i in range(len(theta_ref)):
    theta_plus = theta_ref.copy()
    theta_plus[i] += eps
    y_plus = solve_biofilm(theta_plus, config_short)
    S_fd[:, i] = (y_plus - y_base) / eps

# TSM for same config
S_tsm_short = compute_tsm_sensitivity(theta_ref, config_short)

# Compare
relative_error = np.abs(S_tsm_short - S_fd) / (np.abs(S_fd) + 1e-10)
mean_error = np.mean(relative_error)

print(f"\n✓ Mean relative error: {mean_error:.6f}")
print(f"  TSM and FD agree within {mean_error*100:.2f}%")

## Summary

### Key Takeaways

1. **TSM is fast**: Analytical sensitivities avoid expensive finite differences
2. **Parameter importance**: Some parameters have much stronger influence
3. **Time-varying**: Sensitivity changes over the simulation time
4. **Validation**: TSM agrees with finite differences

### Applications

- **Prior design**: Focus on most influential parameters
- **Experiment design**: Measure outputs most sensitive to unknowns
- **Model reduction**: Identify negligible parameters

### Next Steps

- Try `03_custom_priors.ipynb` to use sensitivity information
- Explore `04_uncertainty_quantification.ipynb` for UQ workflows